In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sympy import diff
from sympy import symbols
from scipy.optimize import fsolve
import sympy as sp
import matplotlib.pyplot as plt

In [2]:
#Teste 1 
# Criação do DataFrame
df = pd.DataFrame({'Cliente': [1, 2], 'X': [1, 0], 'P': [250, 300],'D': [0.4476920904256747, 0.2973393456552685]})

# Definindo as funções
def f(P, X):
    return np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))

def g(P, X):
    return f(P, X) - 0.745031436

def margem(P, X):
    return f(P, X) * (P - 150)

# Função Lagrangiana
def L(P, X, lambd):
    return f(P, X) - lambd * g(P, X)

# Derivada da função Lagrangiana em relação a P e λ
def equations(vars, X):
    P, lambd = vars
    eq1 = f(P, X) - lambd * (f(P, X) - 0.745031436)
    eq2 = g(P, X)
    return [eq1, eq2]

# Resolução para cada cliente
resultados = []
for index, row in df.iterrows():
    X = row['X']
    P_initial = row['P']
    lambd_initial = 1.0
    P_opt, lambd_opt = fsolve(equations, (P_initial, lambd_initial), args=(X))
    resultados.append(P_opt)

# Adicionando os novos valores de P no DataFrame
df['P_otimizado'] = resultados
print(df)




   Cliente  X    P         D  P_otimizado
0        1  1  250  0.447692    -5.578848
1        2  0  300  0.297339   -85.680896


C:\ProgramData\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
C:\ProgramData\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The number of calls to function has reached maxfev = 600.
  warnings.warn(msg, RuntimeWarning)


In [4]:
#Teste 2

import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Criação do DataFrame
df = pd.DataFrame({'Cliente': [1, 2], 'X': [1, 0], 'P': [250, 300],'D': [0.4476920904256747, 0.2973393456552685]})

# Definindo as funções
def f(P, X):
    return np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))

def g(P, X):
    return f(P, X) - 0.745031436

def margem(P, X):
    return f(P, X) * (P - 150)

# Função de otimização
def objetivo(vars, X):
    P, lambd = vars
    return -margem(P, X)  # Negativo porque vamos minimizar

# Restrições
def restricao(vars, X):
    P, lambd = vars
    return g(P, X)

# Resolução para cada cliente
resultados = []
for index, row in df.iterrows():
    X = row['X']
    P_initial = row['P']
    lambd_initial = 1.0
    resultado = minimize(objetivo, 
                         (P_initial, lambd_initial), 
                         args=(X), constraints={'type': 'eq', 'fun': restricao, 'args': [X]},
                         bounds=[(100, 500), (0, None)])
    P_opt, lambd_opt = resultado.x
    resultados.append(P_opt)



In [5]:
# Adicionando os novos valores de P no DataFrame
df['P_otimizado'] = resultados
print(df)

import pandas as pd

df = pd.DataFrame({'Cliente': [1, 2], 'X': [1, 0], 'P': [250, 300],'D': [0.4476920904256747, 0.2973393456552685], 'P_otimizado': [100.0, 100.0]})
print(df)


   Cliente  X    P         D  P_otimizado
0        1  1  250  0.447692        100.0
1        2  0  300  0.297339        100.0
   Cliente  X    P         D  P_otimizado
0        1  1  250  0.447692        100.0
1        2  0  300  0.297339        100.0


In [13]:
#Teste 3 

import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Criação do DataFrame
df = pd.DataFrame({'Cliente': [1, 2], 'X': [1, 0], 'P': [250, 300],'D': [0.4476920904256747, 0.2973393456552685]})

# Definindo as funções
def f(P, X):
    return np.sum(np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P)))

def g(P, X):
    return np.sum(f(P, X) - 0.745031436)

def margem(P, X):
    return sum(f(P, X) * (P - 150))

# Função de Lagrange para otimização
def lagrange(vars, X):
    P, lambd = vars
    return -np.sum((f(P, X) * (P - 150)))  # Negativo porque vamos minimizar

# Restrição
def constraint(vars, X):
    P, lambd = vars
    return np.sum(f(P, X) - 0.745031436)

# Resolução para cada cliente
resultados = []
lambdas = []
for index, row in df.iterrows():
    X = row['X']
    P_initial = row['P']
    lambd_initial = 1.0
    result = minimize(lagrange, (P_initial, lambd_initial), args=(X), constraints={'type': 'eq', 'fun': constraint, 'args': [X]}, bounds=[(100, 500), (None, None)])
    P_opt, lambd_opt = result.x
    resultados.append(P_opt)
    lambdas.append(lambd_opt)




In [14]:
# Adicionando os novos valores de P e λ no DataFrame
df['P_otimizado'] = resultados
df['lambda'] = lambdas
print(df)

   Cliente  X    P         D  P_otimizado  lambda
0        1  1  250  0.447692        100.0     1.0
1        2  0  300  0.297339        100.0     1.0


In [54]:
x = symbols('x')
p1 = symbols('p1')
p2 = symbols('p2')
L = symbols('L')
f=(((sp.exp(1.04)*sp.exp(-0.005*p1))/(1+sp.exp(1.04)*sp.exp(-0.005*p1)))*(p1-150)+((sp.exp(0.64)*sp.exp(-0.005*p2))/(1+sp.exp(0.64)*sp.exp(-0.005*p2)))*(p2-150))-L*((((sp.exp(1.04)*sp.exp(-0.005*p1))/(1+sp.exp(1.04)*sp.exp(-0.005*p1)))+((sp.exp(0.64)*sp.exp(-0.005*p2))/(1+sp.exp(0.64)*sp.exp(-0.005*p2))))-0.7450314360809431)
f

-L*(-0.745031436080943 + 1.89648087930495*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 2.82921701435156*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1))) + 1.89648087930495*(p2 - 150)*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 2.82921701435156*(p1 - 150)*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1))

In [55]:
df_dL = diff(f, L)
df_dp1 = diff(f, p1)
df_dp2 = diff(f, p2)
print(df_dL)
print(df_dp1)
print(df_dp2)

0.745031436080943 - 1.89648087930495*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) - 2.82921701435156*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1))
-L*(-0.0141460850717578*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 0.005*exp(-0.01*p1)/(0.35345468195878 + exp(-0.005*p1))**2) - 0.0141460850717578*(p1 - 150)*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 2.82921701435156*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 0.005*(p1 - 150)*exp(-0.01*p1)/(0.35345468195878 + exp(-0.005*p1))**2
-L*(-0.00948240439652476*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 0.005*exp(-0.01*p2)/(0.527292424043049 + exp(-0.005*p2))**2) - 0.00948240439652476*(p2 - 150)*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 1.89648087930495*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 0.005*(p2 - 150)*exp(-0.01*p2)/(0.527292424043049 + exp(-0.005*p2))**2


In [29]:
print(df_dp1)

L*(-0.0141460850717578*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 0.005*exp(-0.01*p1)/(0.35345468195878 + exp(-0.005*p1))**2) - 0.0141460850717578*(p1 - 150)*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 2.82921701435156*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 0.005*(p1 - 150)*exp(-0.01*p1)/(0.35345468195878 + exp(-0.005*p1))**2


In [32]:
def dp1(p1,L):
    return L*(-0.0141460850717578*np.exp(-0.005*p1)/(1 + 2.82921701435156*np.exp(-0.005*p1)) + 0.005*np.exp(-0.01*p1)/(0.35345468195878 + np.exp(-0.005*p1))**2) - 0.0141460850717578*(p1 - 150)*np.exp(-0.005*p1)/(1 + 2.82921701435156*np.exp(-0.005*p1)) + 2.82921701435156*np.exp(-0.005*p1)/(1 + 2.82921701435156*np.exp(-0.005*p1)) + 0.005*(p1 - 150)*np.exp(-0.01*p1)/(0.35345468195878 + np.exp(-0.005*p1))**2

In [53]:
dp1(287.59604342,196.73901963)

-8.131287310142454e-12

In [34]:
print(df_dp2)

L*(-0.00948240439652476*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 0.005*exp(-0.01*p2)/(0.527292424043049 + exp(-0.005*p2))**2) - 0.00948240439652476*(p2 - 150)*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 1.89648087930495*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 0.005*(p2 - 150)*exp(-0.01*p2)/(0.527292424043049 + exp(-0.005*p2))**2


In [39]:
def dp2(p2,L):
    return L*(-0.00948240439652476*np.exp(-0.005*p2)/(1 + 1.89648087930495*np.exp(-0.005*p2)) + 0.005*np.exp(-0.01*p2)/(0.527292424043049 + np.exp(-0.005*p2))**2) - 0.00948240439652476*(p2 - 150)*np.exp(-0.005*p2)/(1 + 1.89648087930495*np.exp(-0.005*p2)) + 1.89648087930495*np.exp(-0.005*p2)/(1 + 1.89648087930495*np.exp(-0.005*p2)) + 0.005*(p2 - 150)*np.exp(-0.01*p2)/(0.527292424043049 + np.exp(-0.005*p2))**2

In [46]:
dp2(257.783316,196.73901963)

-1.2279233185807925e-11

In [42]:
print(df_dL)

-0.745031436080943 + 1.89648087930495*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 2.82921701435156*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1))


In [43]:
def dpL(p1,p2):
    return -0.745031436080943 + 1.89648087930495*np.exp(-0.005*p2)/(1 + 1.89648087930495*np.exp(-0.005*p2)) + 2.82921701435156*np.exp(-0.005*p1)/(1 + 2.82921701435156*np.exp(-0.005*p1))

In [44]:
dpL(287.59604342,257.783316)

-2.7923219292347312e-12

In [22]:
df_dp1

L*(-0.0141460850717578*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 0.005*exp(-0.01*p1)/(0.35345468195878 + exp(-0.005*p1))**2) - 0.0141460850717578*(p1 - 150)*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 2.82921701435156*exp(-0.005*p1)/(1 + 2.82921701435156*exp(-0.005*p1)) + 0.005*(p1 - 150)*exp(-0.01*p1)/(0.35345468195878 + exp(-0.005*p1))**2

In [24]:
df_dp2

L*(-0.00948240439652476*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 0.005*exp(-0.01*p2)/(0.527292424043049 + exp(-0.005*p2))**2) - 0.00948240439652476*(p2 - 150)*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 1.89648087930495*exp(-0.005*p2)/(1 + 1.89648087930495*exp(-0.005*p2)) + 0.005*(p2 - 150)*exp(-0.01*p2)/(0.527292424043049 + exp(-0.005*p2))**2

In [45]:
def func(X):
    p1 = X[0]
    p2 = X[1]
    L  = X[2] # this is the multiplier. lambda is a reserved keyword in python
    return (((sp.exp(1.04)*sp.exp(-0.005*p1))/(1+sp.exp(1.04)*sp.exp(-0.005*p1)))*(p1-150)+((sp.exp(0.64)*sp.exp(-0.005*p2))/(1+sp.exp(0.64)*sp.exp(-0.005*p2)))*(p2-150))-L*((((sp.exp(1.04)*sp.exp(-0.005*p1))/(1+sp.exp(1.04)*sp.exp(-0.005*p1)))+((sp.exp(0.64)*sp.exp(-0.005*p2))/(1+sp.exp(0.64)*sp.exp(-0.005*p2))))-0.7450314360809431)


def dfunc(X):
    dLambda = np.zeros(len(X))
    h = 1e-3 # this is the step size used in the finite difference.
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X+dX)-func(X-dX))/(2*h);
    return dLambda

from scipy.optimize import fsolve

#this is the max
X1 = fsolve(dfunc, [400, 500, 10])
print(X1) ,
print(func(X1))

[ 287.59604342  257.783316   -196.73901962]
92.2806443407022


In [103]:
#teste iterando
X =[250, 300]
L = [0,0]


def func(X,L):
    p = X
    L  = L # this is the multiplier. lambda is a reserved keyword in python
    #return sp.sum((((sp.exp(0.64)*sp.exp(-0.005*p))/(1+sp.exp(0.64)*sp.exp(-0.005*p)) )*(p-150)-L*(((sp.exp(0.64)*sp.exp(-0.005*p))/(1+sp.exp(0.64)*sp.exp(-0.005*p)) ) -0.7450314360809431))

    return (((sp.exp(0.64)*sp.exp(-0.005*p))/(1+sp.exp(0.64)*sp.exp(-0.005*p)))*(p-150))-L*(((sp.exp(0.64)*sp.exp(-0.005*p))/(1+sp.exp(0.64)*sp.exp(-0.005*p)))-0.7450314360809431)


def dfunc(X,L):
    dLambda = np.zeros(len(X))
    h = 1e-3 # this is the step size used in the finite difference.
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X[i]+dX[i])-func(X[i]-dX[i]))/(2*h);
    return dLambda

from scipy.optimize import fsolve

#this is the max
X1 = fsolve(dfunc, X)
print(X1) ,
print(func(X1))

TypeError: dfunc() missing 1 required positional argument: 'L'

In [79]:
X[0]

250

In [89]:
h = 1e-3
dX = np.zeros(len(X))
dLambda = np.zeros(len(X))

In [90]:
def func(X):
    #p = X
    #L  = L # this is the multiplier. lambda is a reserved keyword in python
    #return sp.sum((((sp.exp(0.64)*sp.exp(-0.005*p))/(1+sp.exp(0.64)*sp.exp(-0.005*p)) )*(p-150)-L*(((sp.exp(0.64)*sp.exp(-0.005*p))/(1+sp.exp(0.64)*sp.exp(-0.005*p)) ) -0.7450314360809431))

    return (( (sp.exp(0.64)*sp.exp(-0.005*X))))

In [91]:
dX

array([0., 0.])

In [92]:
dX[0] = h

In [94]:
dLambda[0] = (func(X[0]+dX[0])-func(X[0]-dX[0]))/(2*h);
dLambda[0]

-0.00271675434543317

In [56]:
from scipy.optimize import minimize
import numpy as np

# Definindo a função objetivo
def objective_function(vars):
    x, y = vars
    return -(x + y)  # Negativo porque a função minimize busca o mínimo

# Definindo a restrição de igualdade para a condição de Lagrange
def constraint(vars):
    x, y = vars
    return x**2 + y**2 - 1

# Restrição como um dicionário
con = {'type': 'eq', 'fun': constraint}

# Chute inicial (ponto próximo ao esperado)
x0 = np.array([0.5, 0.5])

# Executando a otimização
solution = minimize(objective_function, x0, constraints=con)

# Extraindo o resultado
x, y = solution.x
max_value = -solution.fun  # Convertendo de volta para o valor máximo

print(f"Ponto de máximo: (x={x}, y={y})")
print(f"Valor máximo: {max_value}")

# Para encontrar o mínimo, podemos simplesmente mudar o sinal da função objetivo
def objective_function_min(vars):
    x, y = vars
    return x + y

# Executando a otimização para o mínimo
solution_min = minimize(objective_function_min, x0, constraints=con)

# Extraindo o resultado
x_min, y_min = solution_min.x
min_value = solution_min.fun

print(f"Ponto de mínimo: (x={x_min}, y={y_min})")
print(f"Valor mínimo: {min_value}")

Ponto de máximo: (x=0.7071067811873654, y=0.7071067811873659)
Valor máximo: 1.4142135623747314
Ponto de mínimo: (x=0.7071067840613551, y=0.7071067840613549)
Valor mínimo: 1.41421356812271


In [106]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Criando o DataFrame
df = pd.DataFrame({'Cliente': [1, 2], 'X': [1, 0], 'P': [250, 300]})

# Definindo a função f(P)
def f(P, X):
    return np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))

# Aplicando a função f(P) para calcular a demanda estimada D
df['D'] = f(df['P'], df['X'])

# Definindo a função objetivo
def objective_function(P):
    X = df['X']
    return -np.sum(f(P, X) * (P - 150))

# Definindo a restrição g(P)
def constraint(P):
    X = df['X']
    return np.sum(f(P, X) - 0.745031436)

# Restrição como um dicionário
con = {'type': 'eq', 'fun': constraint}

# Chute inicial para os valores de P
P_initial = df['P'].to_numpy()

# Executando a otimização
solution = minimize(objective_function, P_initial, constraints=con)

# Verificando se a otimização foi bem-sucedida
if solution.success:
    # Extraindo os valores otimizados de P
    optimized_P = solution.x
    # Adicionando ao DataFrame
    df['Optimized_P'] = optimized_P
    print(df)
else:
    print("A otimização não foi bem-sucedida. Mensagem:", solution.message)

   Cliente  X    P         D  Optimized_P
0        1  1  250  0.447692   -16.774558
1        2  0  300  0.297339   -76.394355


In [58]:
# Criando o DataFrame
df = pd.DataFrame({'Cliente': [1, 2], 'X': [1, 0], 'P': [250, 300]})

# Definindo a função f(P)
def f(P, X):
    return np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))

# Aplicando a função f(P) para calcular a demanda estimada D
df['D'] = f(df['P'], df['X'])

In [59]:
df

,Cliente,X,P,D
0,1,1,250,0.447692
1,2,0,300,0.297339


In [60]:
0.447692 + 0.297339

0.745031

array([0, 0, 1, 0, 0, 1, 1, 1, 1, 0])

In [ ]:
#https://github.com/ikatsov/tensor-house/blob/master/pricing/price-optimization-multiple-time-intervals.ipynb

In [104]:
#
# Imports and settings
#
import sympy as sy
import numpy as np
from tabulate import tabulate
from scipy.optimize import linprog

def tabprint(msg, A):
    print(msg)
    print(tabulate(A, tablefmt="fancy_grid", floatfmt=".2f"))

In [105]:
import numpy as np
from tabulate import tabulate
from scipy.optimize import linprog
import scipy.stats as stats 
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.animation as animation

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Criando o DataFrame
df = pd.DataFrame({'Cliente': [1, 2], 'X': [1, 0], 'P': [250, 300]})

# Definindo a função f(P)
def f(P, X):
    return np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))

# Aplicando a função f(P) para calcular a demanda estimada D
df['D'] = f(df['P'], df['X'])

# Definindo a função objetivo
def objective_function(P):
    X = df['X']
    return -np.sum(f(P, X) * (P - 150))

# Definindo a restrição g(P)
def constraint(P):
    X = df['X']
    return np.sum(f(P, X) - 0.745031436)

# Restrição como um dicionário
con = {'type': 'eq', 'fun': constraint}

# Chute inicial para os valores de P
P_initial = df['P'].to_numpy()

# Executando a otimização
solution = minimize(objective_function, P_initial, constraints=con)

# Verificando se a otimização foi bem-sucedida
if solution.success:
    # Extraindo os valores otimizados de P
    optimized_P = solution.x
    # Adicionando ao DataFrame
    df['Optimized_P'] = optimized_P
    print(df)
else:
    print("A otimização não foi bem-sucedida. Mensagem:", solution.message)

In [ ]:
###############################################################

In [107]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [108]:
# Carregar a planilha
file_path = 'C:/Users/flcoelho/Desktop/Projetos/Otim/df_teste2.xlsx'
df = pd.read_excel(file_path)

In [109]:
df

,cliente,custo,preco_base
0,1,859.478267,1443.692498
1,2,1001.818691,2108.648054
2,3,2326.178906,3908.887424
3,4,505.110481,808.047482
4,5,1373.205312,2346.958318
5,6,5997.046977,11642.490730
6,7,1405.580425,2444.062641
7,8,765.511459,1546.175438
8,9,940.360500,1504.336106
9,10,883.891280,1683.281814


In [112]:
# Função de demanda para calcular a retenção com base no preço 
def calcular_retencao(P):
    D = np.exp(0.817000384028366 - 0.000338502397377465 * P) / (1 + np.exp(0.817000384028366 - 0.000338502397377465 * P))
    return D


In [113]:
# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(precos):
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada'] = calcular_retencao(precos)

    # Calcular a margem esperada otimizada
    df['preco_otimizado_esperado'] = precos * df['retencao_otimizada']
    df['margem_esperada_otimizada'] = df['preco_otimizado_esperado'] - df['custo']

    # Calcular a margem esperada total (somatório)
    margem_esperada_total = df['margem_esperada_otimizada'].sum()

    # Maximizar a margem (retornamos o negativo para que o método minimize possa maximizar)
    return -margem_esperada_total

In [114]:
# Definir a restrição: a retenção média deve ser igual à retenção base média 
def restricao(precos):
    # Calcular a retenção otimizada média com os preços dados
    retencao_otimizada_media = calcular_retencao(precos).mean()

    # A restrição é que a retenção média otimizada deve ser igual à base
    return retencao_otimizada_media - df['retencao_base_esperado'].mean()

SyntaxError: invalid syntax (1178728944.py, line 5)

In [ ]:
# Verificar os preços otimizados e a margem esperada otimizada 
df['preco_otimizado'] = resultado.x df['retencao_otimizada'] = calcular_retencao(df['preco_otimizado'])
df['preco_otimizado_esperado'] = df['preco_otimizado'] * df['retencao_otimizada']
df['margem_esperada_otimizada'] = df['preco_otimizado_esperado'] - df['custo']

# Resultados
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun

# Exibir os resultados
print("Preços Otimizados:", precos_otimizados) print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print(df)


In [ ]:
# Carregar a planilha
file_path = 'C:/Users/flcoelho/Desktop/Projetos/Otim/df_teste2.xlsx'
df = pd.read_excel(file_path)

In [ ]:
df['retencao_base_esperado']

In [124]:
# Função de demanda para calcular a retenção com base no preço 
def calcular_retencao(P):
    D = np.exp(0.817000384028366 - 0.000338502397377465 * P) / (1 + np.exp(0.817000384028366 - 0.000338502397377465 * P))
    return D

df['retencao_base_esperado']=calcular_retencao(df['preco_base'])

# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(precos):
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada'] = calcular_retencao(precos)

    # Calcular a margem esperada otimizada
    df['preco_otimizado_esperado'] = precos * df['retencao_otimizada']
    df['margem_esperada_otimizada'] = df['preco_otimizado_esperado'] - df['custo']

    # Calcular a margem esperada total (somatório)
    margem_esperada_total = df['margem_esperada_otimizada'].sum()

    # Maximizar a margem (retornamos o negativo para que o método minimize possa maximizar)
    return -margem_esperada_total

# Definir a restrição: a retenção média deve ser igual à retenção base média 
def restricao(precos):
    # Calcular a retenção otimizada média com os preços dados
    retencao_otimizada_media = calcular_retencao(precos).mean()

    # A restrição é que a retenção média otimizada deve ser igual à base
    return retencao_otimizada_media - df['retencao_base_esperado'].mean()

# Estimativas iniciais (os preços base)
precos_iniciais = df['preco_base'].values

# Definir os limites para os preços (0.9 * preco_base <= preco_otimizado <= 1.1 * preco_base) 
limites = [(0.9 * p, 1.1 * p) for p in precos_iniciais]

# Definir as restrições no formato exigido pelo método 'minimize'
restricoes = ({'type': 'eq', 'fun': restricao})

# Executar a otimização
resultado = minimize(objetivo, precos_iniciais, constraints=restricoes, bounds=limites, method='SLSQP')

# Verificar os preços otimizados e a margem esperada otimizada 
df['preco_otimizado'] = resultado.x 
df['retencao_otimizada'] = calcular_retencao(df['preco_otimizado'])
df['preco_otimizado_esperado'] = df['preco_otimizado'] * df['retencao_otimizada'] 
df['margem_esperada_otimizada'] = df['preco_otimizado_esperado'] - df['custo']

# Resultados
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun

# Exibir os resultados
print("Preços Otimizados:", precos_otimizados) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print(df)


Preços Otimizados: [ 1588.06174758  2319.5128594   3517.9986816    888.85223031
  2472.5325471  10478.241657    2473.81528944  1700.7929818
  1654.7697166   1851.6099954   2233.7332402   3997.0070568
  2197.811       1711.5171238   2417.8962148   3295.727865
  7056.4059      3629.2927617   1834.8519695   2474.15911317]
Margem Esperada Total Otimizada: -12876.27309374611
Retenção Média Otimizada: 0.4667572077596948
    cliente        custo    preco_base  retencao_base_esperado  \
0         1   859.478267   1443.692498                0.581347   
1         2  1001.818691   2108.648054                0.525782   
2         3  2326.178906   3908.887424                0.376092   
3         4   505.110481    808.047482                0.632620   
4         5  1373.205312   2346.958318                0.505637   
5         6  5997.046977  11642.490730                0.042128   
6         7  1405.580425   2444.062641                0.497420   
7         8   765.511459   1546.175438                

In [121]:
df

,cliente,custo,preco_base,retencao_base_esperado,retencao_otimizada,preco_otimizado_esperado,margem_esperada_otimizada,preco_otimizado
0,1,859.478267,1443.692498,0.581347,0.569408,904.255631,44.777364,1588.061748
1,2,1001.818691,2108.648054,0.525782,0.507959,1178.218030,176.399339,2319.512859
2,3,2326.178906,3908.887424,0.376092,0.407611,1433.974639,-892.204267,3517.998682
3,4,505.110481,808.047482,0.632620,0.626240,556.635230,51.524749,888.852230
4,5,1373.205312,2346.958318,0.505637,0.495011,1223.930099,-149.275213,2472.532547
5,6,5997.046977,11642.490730,0.042128,0.061231,641.597014,-5355.449963,10478.241657
6,7,1405.580425,2444.062641,0.497420,0.494902,1224.296558,-181.283867,2473.815289
7,8,765.511459,1546.175438,0.572881,0.560029,952.492609,186.981150,1700.792982
8,9,940.360500,1504.336106,0.576343,0.563863,933.064218,-7.296282,1654.769717
9,10,883.891280,1683.281814,0.561489,0.547414,1013.596496,129.705215,1851.609995


In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Carregar a planilha
file_path = 'C:/Users/flcoelho/Desktop/Projetos/Otim/df_teste2.xlsx'
df = pd.read_excel(file_path)

# Expandir o DataFrame para 50.000 registros 
df = pd.concat([df] * 250, ignore_index=True)


# Função de demanda vetorizada para calcular a retenção com base no preço 
def calcular_retencao(P):
    return np.exp(0.817000384028366 - 0.000338502397377465 * P) / (1 + np.exp(0.817000384028366 - 0.000338502397377465 * P))

df['retencao_base_esperado']=calcular_retencao(df['preco_base'])

# Função objetivo vetorizada para maximizar a margem esperada otimizada 
def objetivo(precos):
    # Calcular a retenção otimizada com os preços dados
    retencao_otimizada = calcular_retencao(precos)

    # Calcular a margem esperada otimizada
    preco_otimizado_esperado = precos * retencao_otimizada
    margem_esperada_otimizada = preco_otimizado_esperado - df['custo'].values

    # Maximizar a margem (retornamos o negativo para que o método minimize possa maximizar)
    return -np.sum(margem_esperada_otimizada)

# Definir a restrição vetorizada: a retenção média deve ser igual à retenção base média 
def restricao(precos):
    retencao_otimizada_media = calcular_retencao(precos).mean()
    return retencao_otimizada_media - df['retencao_base_esperado'].mean()

# Estimativas iniciais (os preços base)
precos_iniciais = df['preco_base'].values

# Definir os limites para os preços (0.9 * preco_base <= preco_otimizado <= 1.1 * preco_base) 
limites = [(0.9 * p, 1.1 * p) for p in precos_iniciais]

# Definir as restrições no formato exigido pelo método 'minimize'
restricoes = [{'type': 'eq', 'fun': restricao}]

# Executar a otimização
resultado = minimize(objetivo, precos_iniciais, constraints=restricoes, bounds=limites, method='SLSQP', options={'maxiter': 500})

# Atualizar o DataFrame com os resultados df['preco_otimizado'] = resultado.x df['retencao_otimizada'] = calcular_retencao(df['preco_otimizado'])
df['preco_otimizado_esperado'] = df['preco_otimizado'] * df['retencao_otimizada'] 
df['margem_esperada_otimizada'] = df['preco_otimizado_esperado'] - df['custo']

# Resultados
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun

# Exibir os resultados gerais
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada)
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())


In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import differential_evolution

# Carregar a planilha
file_path = 'C:/Users/flcoelho/Desktop/Projetos/Otim/df_teste2.xlsx'
df = pd.read_excel(file_path)

# Expandir o DataFrame para 50.000 registros df = pd.concat([df] * 2500, ignore_index=True)

# Função de demanda vetorizada para calcular a retenção com base no preço 
def calcular_retencao(P):
    return np.exp(0.817000384028366 - 0.000338502397377465 * P) / (1 + np.exp(0.817000384028366 - 0.000338502397377465 * P))

# Função objetivo vetorizada para maximizar a margem esperada otimizada 
def objetivo(precos):
    # Calcular a retenção otimizada com os preços dados
    retencao_otimizada = calcular_retencao(precos)

    # Calcular a margem esperada otimizada
    preco_otimizado_esperado = precos * retencao_otimizada
    margem_esperada_otimizada = preco_otimizado_esperado - df['custo'].values

    # Maximizar a margem (retornamos o negativo para que o método minimize possa maximizar)
    return -np.sum(margem_esperada_otimizada)

# Estimativas iniciais (os preços base)
precos_iniciais = df['preco_base'].values

# Definir os limites para os preços (0.9 * preco_base <= preco_otimizado <= 1.1 * preco_base) 
limites = [(0.9 * p, 1.1 * p) for p in precos_iniciais]

# Executar a otimização usando o método de Differential Evolution 
resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=1000, popsize=10)

# Atualizar o DataFrame com os resultados 
df['preco_otimizado'] = resultado.x 
df['retencao_otimizada'] = calcular_retencao(df['preco_otimizado'])
df['preco_otimizado_esperado'] = df['preco_otimizado'] * df['retencao_otimizada'] 
df['margem_esperada_otimizada'] = df['preco_otimizado_esperado'] - df['custo']

# Resultados
margem_esperada_total_otimizada = -resultado.fun

# Exibir os resultados gerais
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada)
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())


Margem Esperada Total Otimizada: -12590.900944070188
Retenção Média Otimizada: 0.45414296919103936


In [2]:
df

,cliente,custo,preco_base,preco_otimizado,retencao_otimizada,preco_otimizado_esperado,margem_esperada_otimizada
0,1,859.478267,1443.692498,1588.061748,0.569408,904.255631,44.777364
1,2,1001.818691,2108.648054,2319.512859,0.507959,1178.218030,176.399339
2,3,2326.178906,3908.887424,4299.776166,0.345589,1485.956542,-840.222364
3,4,505.110481,808.047482,888.852230,0.626240,556.635230,51.524749
4,5,1373.205312,2346.958318,2581.654150,0.485780,1254.115700,-119.089612
5,6,5997.046977,11642.490730,10478.241657,0.061231,641.597014,-5355.449963
6,7,1405.580425,2444.062641,2688.468905,0.476754,1281.737162,-123.843263
7,8,765.511459,1546.175438,1700.792982,0.560029,952.492609,186.981150
8,9,940.360500,1504.336106,1654.769717,0.563863,933.064218,-7.296282
9,10,883.891280,1683.281814,1851.609995,0.547414,1013.596496,129.705215


In [ ]:
#df = pandas.read_sas('some_file.sas7bdat')
#print(df.head())

In [1]:
#import pyreadstat
#filename = 'some_file.sas7bdat'
#df, _ = pyreadstat.read_sas7bdat(filename)

ModuleNotFoundError: No module named 'pyreadstat'

In [6]:
import pandas as pd
from scipy.optimize import minimize

# Carregar os dados do arquivo Excel
# Carregar a planilha
file_path = 'C:/Users/flcoelho/Desktop/Projetos/Otim/analise_teste.xlsx'
df = pd.read_excel(file_path)

In [7]:
import pandas as pd
from scipy.optimize import minimize

# Carregar a planilha
file_path = 'C:/Users/flcoelho/Desktop/Projetos/Otim/analise_teste.xlsx'
df = pd.read_excel(file_path)

# Definir a função objetivo para maximizar o NPV (Valor Presente Líquido)
def objective_function(p, df):
    # Calcular o NPV para cada cliente
    df['NPV'] = df['Act_cost'] * p
    # Somar o NPV de todos os clientes para obter o NPV total
    total_npv = df['NPV'].sum()
    # Como estamos usando a função minimize, retornamos o NPV negativo para maximizar
    return -total_npv

# Definir as restrições para o problema de otimização
def constraint_function(p, df, c):
    # Calcular o valor da restrição
    constraint_value = df['Act_cost'].sum() * (p - c)
    return constraint_value

# Definir os limites para o problema de otimização
lb = 0.9  # Limite inferior
ub = 1.1  # Limite superior
bounds = [(lb, ub)]

# Chute inicial para o preço
initial_guess = [1.0]

# A constante 'c' para a equação de restrição g = c
c = df['Act_cost'].sum()  # Valor exemplo para 'c'

# Definir as restrições em formato de dicionário para o otimizador
constraints = [{'type': 'eq', 'fun': constraint_function, 'args': (df, c)}]

# Realizar a otimização
result = minimize(
    fun=objective_function,
    x0=initial_guess,
    args=(df,),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Extrair o preço otimizado
optimized_price = result.x[0]

# Imprimir os resultados
print(f"O preço otimizado é: {optimized_price}")
print(f"O NPV máximo é: {-result.fun}")

O preço otimizado é: 1.0092238204784543
O NPV máximo é: 155254.12317442326
